# Evaluation with scikit-learn

**CS5483 Data Warehousing and Data Mining**
___

Before we begin coding, it is useful to execute the [line magic](https://ipython.readthedocs.io/en/stable/interactive/magics.html#line-magics) to initialize the environment, and import the libraries necessary for the notebook.

In [ ]:
%reset -f
%matplotlib inline
import numpy as np
from IPython import display
from ipywidgets import interact, IntSlider
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_predict, cross_val_score, StratifiedKFold
from sklearn import tree
from functools import lru_cache

## Data Preparation

### About the dataset

We will use a popular dataset called the [*iris dataset*](https://en.wikipedia.org/wiki/Iris_flower_data_set). Iris is a flower with three different species shown below.

<span style="text-align: center;">
    <div style="display: flex;">
        <div style="padding: 1em;">
            <img src="https://upload.wikimedia.org/wikipedia/commons/5/56/Kosaciec_szczecinkowaty_Iris_setosa.jpg" alt="Iris Setosa" style="width: 300px;">
            Iris Setosa
        </div>
        <div style="padding: 1em;">
            <img src="https://upload.wikimedia.org/wikipedia/commons/4/41/Iris_versicolor_3.jpg" alt="Iris Versicolor" style="width: 300px;">
            Iris Versicolor
        </div>
        <div style="padding: 1em;">
            <img src="https://upload.wikimedia.org/wikipedia/commons/9/9f/Iris_virginica.jpg" alt="Iris Virginica" style="width: 300px;">
            Iris Virginica
        </div>
    </div>
</span>

The three iris species differ in the lengths and widths of their *petals* and *sepals*.

<center><img src="https://upload.wikimedia.org/wikipedia/commons/7/78/Petal-sepal.jpg" style="width: 300px" alt="Petal and Sepal"></center>

A standard data mining task is to train a model that can classify the spieces (*target*) automatically based on the lengths and widths of the petals and sepals (*input features*).

### Load dataset from scikit-learn

**How to load the iris dataset?**

To load the iris dataset, we can simply import the [`sklearn.datasets` package](https://scikit-learn.org/stable/datasets/index.html).

In [ ]:
from sklearn import datasets

iris = datasets.load_iris()
type(iris)  # object type

`sklearn` stores the dataset as a [`Bunch` object](https://scikit-learn.org/stable/modules/generated/sklearn.utils.Bunch.html), which is essentially [a bunch of properties](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_iris.html) put together.

**How to learn more about a library?**

Detailed documentation can be found at <https://scikit-learn.org>.
- The following `IFrame` object embeds the website as an HTML iframe in jupyter notebook. 
- The class `IFrame` is available after importing the `display` module from `IPython` package first.

In [ ]:
from IPython import display
display.IFrame(src="https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_iris.html", width=800, height=600)

We can use the symbol `?` to obtain the docstring of an object and `??` to obtain its source code, if available.

In [ ]:
?datasets.load_iris

In [ ]:
??datasets.load_iris

**How to learn more about the dataset?**

The property `DESCR` (description) is a string that contains some background information of the dataset:

In [ ]:
print(iris.DESCR)

All the properties of an object can be listed using the built-in function `dir` (directory):

In [ ]:
dir(iris)

**How to show the data?**

The properties `data` and `target` contains the data values.

In [ ]:
type(iris.data), type(iris.target)

The data are stored as `numpy` array, which is a powerful data type optimized for performance. It provides useful properties and methods to describe and process the data:

In [ ]:
iris.data.shape, iris.data.ndim, iris.data.dtype

`iris.data` is a 150-by-4 2-dimensional array of 64-bit floating-point numbers.
- 150 corresponds to the number of instances, while
- 4 corresponds to the number of input attributes.

To show the input feature names:

In [ ]:
iris.feature_names

To show the means and standard deviations of the input features:

In [ ]:
iris.data.mean(axis=0), iris.data.std(axis=0)

All the public properties/methods of `numpy` array are printed below:

In [ ]:
import numpy as np
print(*(attr for attr in dir(np.ndarray)
        if attr[0] != '_'))  # private attributes begin with underscore

The above imports `numpy` and rename it as `np` for convenience.

**What is the target feature?**

The target variable of the iris dataset is the flower type, whose names are stored by the following property:

In [ ]:
iris.target_names

`iris.target` is an array of integer indices from `{0, 1, 2}` for the three classes.

In [ ]:
iris.target

**Exercise** Fill the following cell with a tuple of the following properties for the target (instead of input features) of the iris dataset:
- shape, 
- number of dimensions, and 
- the data types of the values.

Your solution should look like:
```Python
iris.___.___, iris.___.___, iris.___.___
```

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
# tests
shape, ndim, dtype = Out[len(In)-2] # retrieve the last output as the answer, 
                     # so execute this only after executing your solution cell
assert isinstance(shape, tuple) and isinstance(ndim, int) and isinstance(dtype, np.dtype)

**Exercise** Fill in the following cell with a tuple of
- the list of minimum values of the input features, and
- the list of maximum values of the input features.

You answer should look like:
```Python
iris.___.___(axis=0), iris.___.___(axis=0)
```

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
# tests
feature_min, feature_max = Out[len(In) - 2]
assert feature_min.shape == (4, ) == feature_max.shape

### Create pandas DataFrame

The [package `pandas`](https://pandas.pydata.org/docs/user_guide/index.html) provides additional tools to display and process a dataset. 
First, we translate the `Bunch` object into a `pandas` [`DataFrame` object](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html?highlight=dataframe#pandas.DataFrame).

In [ ]:
import pandas as pd

# write the input features first
iris_df = pd.DataFrame(data=iris.data, columns=iris.feature_names)

# append the target values to the last column
iris_df['target'] = iris.target

iris_df  # to display the DataFrame

In jupyter notebook, a `DataFrame` object is conveniently displayed as an HTML table, so there is no need `print` it. We can control how much information to show by setting the [display options](https://pandas.pydata.org/pandas-docs/stable/user_guide/options.html).

We can also display the statistics of different numerical attributes using the method `describe` and `boxplot`.

In [ ]:
iris_df.describe()

In [ ]:
%matplotlib inline
iris_df.boxplot(figsize=(10,5)) # figsize specifies figure (width,height) in inches

The line magic [`%matplotlib`](https://ipython.readthedocs.io/en/stable/interactive/magics.html#magic-matplotlib) specifies where the plot should appear.

**How to handle nominal class attribute?**

Note that the boxplot also covers the target attribute, but it should not. (Why?) Let's take a look at the current datatypes of the different attributes.

In [ ]:
print(iris_df.dtypes)

The target is regarded as a numeric attribute with type integer `int64`. Instead, the target should be categorical with only allow three possible values, one for each iris species.

To fix this, we can use the `astype` method to convert the data type automatically. (More details [here](https://pandas.pydata.org/pandas-docs/stable/user_guide/categorical.html#series-creation.section).)

In [ ]:
iris_df.target = iris_df.target.astype('category')
iris_df.boxplot(figsize=(10,5)) # target is not plotted as expected
iris_df.target.dtype

We can also rename the target categories `{0, 1, 2}` to the more meaningful names of the iris species in `iris.target_names`.  
(See the [documentation](https://pandas.pydata.org/pandas-docs/stable/user_guide/categorical.html#renaming-categories).)

In [ ]:
iris_df.target.cat.categories = [iris.target_names[i] for i in range(3)]
iris_df # check that the target values are now setosa, versicolor, or virginica.

**Exercise** For nominal attributes, a more meaningful statistics than the mean is the counts of different possible values. To count the number of instances for each flower class, assign `target_counts` to the output of the `value_counts` method of an appropriate column of `iris_df`.

Your solution should look like:
```Python
target_counts = iris_df.target.___()
```

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()
target_counts

In [ ]:
# tests
assert target_counts.shape == (3, )

**How to select specific rows and columns?**

The following uses [`ipywidget`](https://ipywidgets.readthedocs.io/en/latest/) to show the various ways of selecting/slicing the rows of a `DataFrame`.

In [ ]:
from ipywidgets import interact


@interact(command=[
    'iris_df.head()', 'iris_df[0:4]', 'iris_df.iloc[0:4]', 'iris_df.loc[0:4]',
    'iris_df.loc[iris_df.index.isin(range(0,4))]',
    'iris_df.loc[lambda df: df.target==0]', 'iris_df.tail()', 'iris_df[-1:]'
])
def select_rows(command):
    output = eval(command)
    display.display(output)

The following shows the various ways of slicing different columns.

In [ ]:
@interact(command=[
    'iris_df.target', 'iris_df["target"]', 'iris_df[["target"]]',
    'iris_df[iris_df.columns[:-1]]',
    'iris_df.loc[:,iris_df.columns[0]:iris_df.columns[-1]]',
    'iris_df.loc[:,~iris_df.columns.isin(["target"])]', 'iris_df.iloc[:,:-1]'
])
def select_columns(command):
    output = eval(command)
    display.display(output)

For instance, to compute the mean values of the input features for iris setosa:

In [ ]:
iris_df[lambda df: df.target == 'setosa'].mean()

We can also use the method `groupby` to obtain the mean values by flower types:

In [ ]:
iris_df.groupby(['target']).mean()

**Exercise** Create a new `DataFrame` `iris2d_df`. Note that you may also use the method `drop`.

In [ ]:
# to learn how to use drop
?iris_df.drop

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()
iris2d_df

In [ ]:
# tests
assert set(
    iris2d_df.columns) == {'petal length (cm)', 'petal width (cm)', 'target'}

### Alternatives methods of loading a dataset

The following code loads the iris dataset from an [ARFF file](https://waikato.github.io/weka-wiki/formats_and_processing/arff/) instead.

In [ ]:
from scipy.io import arff
import urllib.request
import io

ftpstream = urllib.request.urlopen(
    'https://raw.githubusercontent.com/Waikato/weka-3.8/master/wekadocs/data/iris.arff'
)
iris_arff = arff.loadarff(io.StringIO(ftpstream.read().decode('utf-8')))
iris_df2 = pd.DataFrame(iris_arff[0])
iris_df2['class'] = iris_df2['class'].astype('category')
iris_df2

In [ ]:
from scipy.io import arff
import urllib.request
import io

ftpstream = urllib.request.urlopen(
    'https://www.openml.org/data/download/61/dataset_61_iris.arff')
iris_arff = arff.loadarff(io.StringIO(ftpstream.read().decode('utf-8')))
iris_df2 = pd.DataFrame(iris_arff[0])
iris_df2['class'] = iris_df2['class'].astype('category')
iris_df2

Pandas also provides a method to read the iris dataset directly from a CSV file locally or from the internet such as the [UCI respository](https://archive.ics.uci.edu/ml/datasets/iris).

In [ ]:
iris_df3 = pd.read_csv(
    'https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data',
    sep=',',
    dtype={'target': 'category'},
    header=None,
    names=iris.feature_names + ['target'],
)
iris_df3

The additional arguments `dtype`, `header`, and `names`, which allow us want specify the attribute datatypes and names. Unlike the ARFF format, CSV file may not contain such information.

## Training and Testing

To give an unbiased performance estimate of a learning algorithm of interest, the fundamental principle is *to use separate datasets for training and testing*.

If there is only one dataset, we should split it into *training sets* and *test sets* by *random sampling* to avoid bias in the performance estimate. In the following subsections, we will illustrate some methods of splitting the datasets for training and testing.

### Stratified holdout method

We randomly samples data for training or testing without replacement. It is implemented by the `train_test_split` function from the `sklearn.model_selection` package.

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(
    iris_df[iris.feature_names],
    iris_df.target,
    test_size=0.2,  # fraction for test
    random_state=1)  # random seed

X_train.shape, X_test.shape, Y_train.shape, Y_test.shape

We also separated the input features and target for the training and test sets. The fraction of holdout test data is

In [ ]:
len(Y_test) / (len(Y_test) + len(Y_train))

The class proportion of the iris dataset is:

In [ ]:
iris_df.target.value_counts().plot(kind='bar', ylabel='counts')

We can check that the class proportions for the test and training sets are maintained in expectation:

In [ ]:
@interact(data=['Y_train', 'Y_test'], seed=(0, 10))
def class_proportions(data, seed=0):
    Y_train, Y_test = train_test_split(iris_df.target,
                                       test_size=0.2,
                                       random_state=seed)
    eval(data).value_counts().sort_index().plot(kind='bar', ylabel='counts')

We first apply a learning algorithm to train a classifier using only the training set. Let's say we want to evaluate the decision tree induction algorithm in `sklearn`.

In [ ]:
from sklearn import tree

clf = tree.DecisionTreeClassifier(random_state=0) # the training is also randomized
clf.fit(X_train, Y_train) # fit the model to the training set 

We can use the `predict` method of the classifier to predict the flower type from input features.

In [ ]:
Y_pred = clf.predict(X_test)

The following code returns the accuracy of the classifier, namely, the fraction of correct predictions on the test set.

In [ ]:
accuracy_holdout = (Y_pred == Y_test).mean()
accuracy_holdout

The `score` method performs the same computation. The following uses f-string to format the accuracy to 3 decimal places.

In [ ]:
print(f'Accuracy: {clf.score(X_test, Y_test):0.3f}')

To see input features of misclassified test instances:

In [ ]:
X_test[Y_pred != Y_test]

**Exercise** Assign `accuracy_holdout_training_set` to the accuracy of the predictions on the training set.  
Note that the accuracy is overly optimistic.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()
accuracy_holdout_training_set

In [ ]:
# hidden tests

**Exercise** Complete the following function which applies random subsampling to reduce the variance of the accuracy estimate. In particular, the function `subsampling_score` should return the average of `N` accuracies of $20\%$ stratified holdout with random seed set from `0` up to `N-1`, where `N` is the integer input argument of the function.

In [ ]:
import numpy as np
from functools import lru_cache

@lru_cache(None) # cache the return value to avoid repeated computation
def holdout_score(seed):
    clf = tree.DecisionTreeClassifier(random_state=seed)
    X_train, X_test, Y_train, Y_test = train_test_split(iris_df[iris.feature_names], 
                                                    iris_df.target, 
                                                    test_size=0.2,
                                                    random_state=seed)
    # YOUR CODE HERE
    raise NotImplementedError()

@lru_cache(None)
def subsampling_score(N):
    return sum(holdout_score(i) for i in range(N))/N

In [ ]:
# tests
assert np.isclose(subsampling_score(50), 0.9466666666666663)

The following code plots the mean accuracies for different `N`. The variance should be smaller as `N` increases.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.stem([subsampling_score(i) for i in range(1,50)])
plt.xlabel(r'$N$')
plt.ylabel(r'Mean accuracy')

The documentation [here](https://ogrisel.github.io/scikit-learn.org/sklearn-tutorial/modules/generated/sklearn.cross_validation.Bootstrap.html) describes  another alternative called the boostrap method, which samples without replacement.

### Stratified cross-validation

Another method of evaluating a classification algorithm is to randomly partition the data into $k$ *folds*, which are nearly equal-sized blocks of instances. The score is the average of the accuracies obtained by using each fold to test a classifier trained using the remaining folds.

<center><img src="https://upload.wikimedia.org/wikipedia/commons/4/4b/KfoldCV.gif" style="width:500px" alt="Cross validation"></center>

The module `sklearn.model_selection` provides two functions `cross_val_predict` and `cross_val_score` for this purpose.

In [ ]:
from sklearn.model_selection import cross_val_predict, cross_val_score, StratifiedKFold
cv = StratifiedKFold(n_splits=5, random_state=0, shuffle=True)

For instance, the following returns the misclassified instances by 5-fold cross-validation.

In [ ]:
iris_df['prediction'] = pd.Categorical(cross_val_predict(clf, iris_df[iris.feature_names], iris_df.target, cv=cv))
iris_df.loc[lambda df: df['target'] != df['prediction']]

In [ ]:
clf = tree.DecisionTreeClassifier(random_state=0)
scores = cross_val_score(clf, iris_df[iris.feature_names], iris_df.target, cv=5)
print('Accuracies: ',', '.join(f'{acc:.4f}' for acc in scores))
print(f'Mean accuracy: {scores.mean():.4f}')

**Exercise** Assign `accuracy_cv` to the accuracy obtained by the cross validation result above.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()
accuracy_cv

In [ ]:
# hidden tests